In [17]:
# Para utilizar las funciones group_by, summarise y %>% 
# Simplifican considerablemente el tratamiento de datos agrupados, evitándose bucles, 
library(dplyr)
library(lubridate)

In [4]:
# Carga del CSV
cdp_raw <- read.csv("./CdP_estadistica.csv")

In [12]:
# colnames(cdp_raw)
cdp_raw$Date_f <- as.Date(cdp_raw$date)

[1] "date"          "AVG.Temp."     "AVG.Press."    "AVG.Cond."    
 [5] "AVG.Salinity." "AVG.DO."       "AVG.rawO2."    "AVG.OxySat."  
 [9] "AVG.pH."       "AVG.Redox."    "date_f"

In [37]:
data_ <- cdp_raw[year(cdp_raw$Date_f) %in% c(2014, 2015) & month(cdp_raw$Date_f) %in% c(7,8,9),]
data_2014 <- cdp_raw[year(cdp_raw$Date_f) %in% c(2014) & month(cdp_raw$Date_f) %in% c(7,8,9),]
data_2015 <- cdp_raw[year(cdp_raw$Date_f) %in% c(2015) & month(cdp_raw$Date_f) %in% c(7,8,9),]

In [38]:
est_2014 <- data.frame(
    mean = mean(data_2014$AVG.Temp),
    sd = sd(data_2014$AVG.Temp),
    var = var(data_2014$AVG.Temp),
    n = length(data_2014$AVG.Temp)
)
est_2015 <- data.frame(
    mean = mean(data_2015$AVG.Temp),
    sd = sd(data_2015$AVG.Temp),
    var = var(data_2015$AVG.Temp),
    n = length(data_2015$AVG.Temp)
)
est_ <- data.frame(
    mean = mean(data_$AVG.Temp),
    sd = sd(data_$AVG.Temp),
    var = var(data_$AVG.Temp),
    n = length(data_$AVG.Temp)
)
est_2014
est_2015
est_

mean,sd,var,n
17.58554,1.896687,3.597423,92


mean,sd,var,n
18.5962,1.903122,3.621872,92


mean,sd,var,n
18.09087,1.961294,3.846672,184


In [ ]:
library(dplyr)
library(lubridate)
datos<-read.csv("CdP_estadistica.csv")
datos<- mutate(datos, year=year(date), month=month(date), yday=yday(date))
datos_verano14<-datos[(datos$month==7 | datos$month==8 | datos$month==9) & datos$year==2014,]
datos_verano15<-datos[(datos$month==7 | datos$month==8 | datos$month==9) & datos$year==2015,]
#Media
med14<-mean(datos_verano14$AVG.Temp.)
med15<-mean(datos_verano15$AVG.Temp.)
med14
med15
#Desviaciones Estándar
desv_est14<-sd(datos_verano14$AVG.Temp.)
desv_est15<-sd(datos_verano15$AVG.Temp.)
desv_est14
desv_est15
#Varianza
varianza14<-var(datos_verano14$AVG.Temp.)
varianza15<-var(datos_verano15$AVG.Temp.)
varianza14
varianza15
#Tests
mitest<-t.test(datos_verano14$AVG.Temp.,datos_verano15$AVG.Temp.)
#Como el p-value es muy cercano a cero, luego rechazo la hipótesis de que las dos
#distribuciones sean la misma
mivartest<-var.test(datos_verano14$AVG.Temp.,datos_verano15$AVG.Temp.)
#El p-value asociado a la varianza es cercano a 1, luego no puedo rechazar la hipotesis de que las
#dos distribuciones sean la misma por varianza.

#Histogramas
hist(datos_verano14$AVG.Temp.)
hist(datos_verano15$AVG.Temp.)

#Las dos distribuciones son distintas, sin embargo el comportamiento es muy similar

datos_verano<-datos[(datos$month==7 | datos$month==8 | datos$month==9),]
#Media
med<-mean(datos_verano$AVG.Temp.)
med
#Desviaciones Estándar
desv_est<-sd(datos_verano$AVG.Temp.)
desv_est
#Varianza
varianza<-var(datos_verano$AVG.Temp.)
varianza


dat1<-read.table("dat1.DAT")
dat2<-read.table("dat2.DAT")
dat11<-dat1[!is.na(dat1)]
dat22<-dat2[!is.na(dat2)]


#Media
mean(dat11)
mean(dat22)

#Desviacion estandar
sd(dat11)
sd(dat22)

#Varianza
var(dat11)
var(dat22)

testdat<-t.test(dat11,dat22)
#p.value=1 Por lo que no podemos descartar que sean diferentes aunque tampoco
#podemos garantizar que sean iguales.
vartestdat<-var.test(dat11,dat22)
#p. value nos sale cercano a uno por lo que no podemos garantizar que se trate de distribuciones
#diferentes , pero tampoco garantizar que sean iguales.

#Histogramas
hist(dat11)
hist(dat22)

#Aunque los test no descarten que las distribuciones sean distintas en el histograma
#podemos ver como en realidad las funciones son muy distintas.

In [41]:
bootstrap <- function(x) {
    sample(x, replace=TRUE)
}

nbootstrap <- function(x, n) {
    m <- matrix(0, ncol=n, nrow=length(x))
    for (i in 1:n) {
        m[,i] <- bootstrap(x)
    }
    m
}

In [56]:
M <- 100
data_boot <- nbootstrap(data_$AVG.Temp, M)
data_boot_m <- c()
for (i in 1:M)
    data_boot_m <- c(data_boot_m, mean(data_boot[,i]))

In [57]:
data.frame(
    media = mean(data_boot_m),
    sd = sd(data_boot_m),
    sesgo = 1/M * sum(data_boot_m - est_$mean)
)

media,sd,sesgo
18.08265,0.1416633,-0.008221196


In [52]:
jackknife <- function(x) {
    m <- matrix(0, ncol=length(x), nrow=length(x)-1)
    for (i in 1:length(x)) {
        m[,i] <- x[-i]
    }
    m
}

In [54]:
data_jack <- jackknife(data_$AVG.Temp)
data_jack_m <- c()
for (i in 1:M)
    data_jack_m <- c(data_jack_m, mean(data_jack[,i]))

In [55]:
data.frame(
    media = mean(data_jack_m),
    sd = sd(data_jack_m),
    sesgo = 1/M * sum(data_jack_m - est_$mean)
)

media,sd,sesgo
18.09298,0.01021415,0.00210967
